# Laboratorio de regresión - 3

## Significancia de factores

|                |   |
:----------------|---|
| **Nombre**     | Pablo Josué Panécatl García  |
| **Fecha**      | 25/08/25  |
| **Expediente** | 751773  |

Descarga el archivo de publicidad y carga los datos (Advertising.csv).

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [14]:
datos = pd.read_csv(r"C:\Users\pablo\OneDrive - ITESO\Semestre 5\Laboratorio de aprendizaje estadístico\Advertising.csv")

In [15]:
from sklearn.linear_model import LinearRegression

**¿Hay alguna relación entre el presupuesto para publicidad y las ventas?**

Nuestra primera meta debe ser determinar si hay evidencia en los datos de que haya una asociación entre estas variables.

- ¿Por qué? ¿Qué resultaría si nos diéramos cuenta de la falta de relación entre el presupuesto de publicidad y las ventas?

**¿Qué tan fuerte es esta relación?**
Asumiendo que existe esta relación, ¿nos sirve conocer el impacto que tiene invertir en publicidad en las ventas?

**¿Cuáles medios están asociados con las ventas? ¿Qué tan grande es la asociación entre un medio específico y las ventas?**

Hay 3 medios distintos en los datos. ¿Sirve invertir en los 3? ¿Conviene más invertir sólo en uno?

**¿Qué tan seguros estamos de que podríamos predecir ventas futuras?**

**¿La relación es lineal?**

**¿Hay sinergia entre estos medios?**

Puede ser que gastar \\$50,000 en publicidad y otros \\$50,000 en radio es mejor opción que gastar \\$100,000 en televisión. A esto le llamamos efecto de interacción.

Podemos usar regresión lineal para responder todas estas preguntas.

Realiza una regresión lineal:

$$ \text{ventas} \approx \beta_0 + (\beta_1)(\text{TV})$$

In [16]:
X = datos.TV.values.reshape(-1, 1)
y = datos.sales.values

In [17]:
lr = LinearRegression()
lr.fit(X, y)
lr.coef_, lr.intercept_

(array([0.04753664]), 7.032593549127694)

In [18]:
beta0 = lr.intercept_
beta1 = lr.coef_[0]

### Verificando la precisión de nuestros coeficientes estimados

Recuerda que en el mundo real hay ruidos y errores de medición. Siempre se asume que la verdadera relación entre $X$ y $Y$ es $$Y = \beta_0 + \beta_1 X + \epsilon$$

Se asume que el término de error es independiente de $X$ (el error siempre es el mismo sin importar el valor de $X$). Este modelo describe a la *línea de regresión de la población*, que es la mejor aproximación de la verdadera relación entre $X$ y $Y$. Cuando usamos mínimos cuadrados encontramos la *línea de mínimos cuadrados*.

¿Cuál es la diferencia entre población y muestra?

Población es la cantidad total de datos que tenemos mientras que la muestra es una pequeña parte de la población, apartada para analizarla

¿Cuál crees que sea la diferencia entre hacer una regresión con todos los datos de la población y una muestra de ella?

La regresión para la población completa sería 100% verdadera mientras la de la muestra se aproxima al 100

La línea de regresión de la población no se puede observar. El concepto de comparar estas líneas es una extensión natural del acercamiento estadístico estándar de usar información de una muestra para estimar características de una población grande.

Imagina que quieres encontrar la altura promedio de un mexicano $\mu$. Medir a todos y cada uno de los mexicanos en situaciones similares, con la misma regla, mismo operador, y otras incontables formas de minimizar la variación de la medida es una tarea imposible. Lo que podemos asumir es que $\hat{\mu} = \bar{y}$. La media poblacional y la media muestral son diferentes, pero la media muestral es usualmente un buen estimado.

De la misma manera, como no contamos con el 100% de la información para hacer una regresión, los coeficientes $\beta_0$ y $\beta_1$ son desconocidos. Podemos estimarlos usando mínimos cuadrados, encontrando $\hat{\beta_0}$ y $\hat{\beta_1}$. Puede que las muestras que tengamos en ese momento estén un poco por encima de la media, pero otras muestras en otro momento puede que estén debajo de la media. En general, esperamos que el promedio de las aproximaciones $\hat{\mu}$ aproxime a $\mu$.

Esto lleva a la pregunta: ¿qué tan cercanos son nuestros coeficientes estimados a los verdaderos coeficientes? Utilizamos el concepto de error estándar para evaluar esto.

$$ \text{Var}(\hat{\mu})=\text{SE}(\hat{\mu})^2 = \frac{\sigma^2}{n} $$

Donde $\sigma$ es la desviación estándar de cada una de las observaciones $y_i$ de $Y$. El error estándar nos dice la cantidad promedio que el estimado difiere del valor verdadero. Podemos ver en la fórmula que entre más observaciones tengamos el error se hace más pequeño. Las fórmulas para errores estándar de $\hat{\beta_0}$ y $\hat{\beta_1}$ son:

$$ \text{SE}(\hat{\beta_0})^2 = \sigma^2 [\frac{1}{n} + \frac{\bar{x}^2}{\sum_{i=1}^n (x_i - \bar{x})^2}]$$

$$ \text{SE}(\hat{\beta_1})^2 = \frac{\sigma^2}{\sum_{i=1}^n (x_i - \bar{x})^2}$$

$$ \sigma^2 = \text{Var}(\epsilon) = \text{RSE}^2 = \frac{\text{RSS}}{n-p}$$

Para que estas fórmulas sean validas asumimos que los errores $\epsilon_i$ tienen varianza común $\sigma^2$ y que no están correlacionados.

Calcula los errores estándar de los coeficientes

In [19]:
n = datos.sales.count()
denom = (datos.TV - datos.TV.mean())**2
denom = denom.sum()
SE_beta0 = var*(1/n + (datos.TV.mean()**2)/denom)
SE_beta0 = np.sqrt(SE_beta0)
SE_beta0

0.45784294027347855

In [20]:
SE_beta1 = var/denom
SE_beta1 = np.sqrt(SE_beta1)
SE_beta1

0.0026906071877968707

In [21]:
RSS = ((lr.predict(X) - y) ** 2).sum()
RSS
var = RSS/(n-2)
var

10.618841328946221

In [22]:
upper = beta0 + SE_beta0 * 2
upper

7.948279429674651

In [23]:
lower = beta0 - SE_beta0 * 2
lower

6.116907668580737

Estos errores se pueden usar para calcular intervalos de confianza. Un intervalo de confianza del $95\%$ se define como un rango de valores en el cuál se encuentra el desconocido valor verdadero con un $95\%$ de probabilidad.

Otra forma de verlo es que si tomamos muestras repetidas y construimos un intervalo de confianza para cada una, el $95\%$ de los intervalos creados van a contener el valor verdadero. Para la regresión el intervalo de confianza del $95\%$ toma la forma:

$$ \hat{\beta_j} \pm 2\text{SE}(\hat{\beta_j})$$

Calcula los intervalos de confianza para los coeficientes estimados:

In [24]:
upper1 = beta1 + 2*SE_beta1
upper1

0.052917854808613496

In [25]:
lower1 = beta1 - 2*SE_beta1
lower1

0.04215542605742602

Los errores estándar también se usan para realizar pruebas de hipótesis. La prueba de hipótesis más común es probar la hipótesis nula de:

$$ H_0: \text{No hay relación entre } X \text{ y } Y \ \ \ \ (\beta_1=0)$$

contra la hipótesis alternativa:
$$ H_0: \text{Hay alguna relación entre } X \text{ y } Y \ \ \ (\beta_1 \neq 0)$$

Explica con tus palabras el significado de la hipótesis nula y la hipótesis alternativa.

La hipótesis nula supone que no hay ninguna relación entre el presupuesto para publicidad en TV y las ventas totales

La hipótesis alterna supone que debe haber algún tipo de relación entre estas dos

Para probal la hipótesis nula debemos determinar si nuestro estimado $\hat{\beta_1}$ de $\beta_1$ está lo suficientemente alejado de cero para que podamos decir con confianza que este valor no es cero. 

¿Qué tan lejos? Depende de qué tanta confianza tengamos en el estimado encontrado. Si nuestro error estándar es pequeño y nuestro estimado está alejado de cero podríamos decir que hay muy poca probabilidad de que el valor verdadero sea 0. En cambio, si nuestro error estándar es grande y nuestro estimado está muy cerca de cero, entonces podrías ser que el valor verdadero sea cero y que no haya relación entre las variables.

Se calcula un *estadístico t* dado por
$$ t = \frac{\hat{\beta_j} - \mu}{\text{SE}(\hat{\beta_j})} $$

donde $\mu$ es el valor contra el que queremos probar.

Calcula el estadístico t para tus coeficientes estimados, usando como referencia la prueba de hipótesis.

In [26]:
t1 = beta1/SE_beta1
t1

17.66762560087555

In [27]:
t0 = beta0/SE_beta0
t0

15.360275174117543

La distribución t tiene forma de campana y se parece bastante a la distribución normal cuando $n > 30$. Ya sólo es cuestión de calcular la probabilidad de observar cualquier número tal que su valor absoluto sea igual o mayor que el valor absoluto del estadístico t calculado. En otras palabras:
$$ P(|x| \geq |t|) $$

A esta probabilidad la llamamos *p-value*. Un *p-value* pequeño indica que es poco probable que exista por puro azar una relación significativa entre predictor y respuesta, en caso de que no haya una asociación real entre predictor y respuesta. En otras palabras, el *p-value* te dice la probabilidad de que parezca que hay relación cuando no la hay.

Si el *p-value* es pequeño, inferimos que sí hay una asociación entre el predictor y la respuesta, y **rechazamos la hipótesis nula**.
  

¿Qué tan pequeño? Depende de la aplicación. Un valor muy común es del $5\%$.

Utiliza el siguiente código para calcular el *p-value* para tus coeficientes

`from scipy import stats`

`p_bj = 2*(1 - stats.t.cdf(np.abs(t_bj), n-p))`

In [28]:
from scipy import stats

In [29]:
p_b1 = 2*(1 - stats.t.cdf(np.abs(t1), df=n-2))
p_b1

0.0

In [30]:
p_b0 = 2*(1 - stats.t.cdf(np.abs(t0), df=n-2))
p_b0

0.0

¿Se rechaza la hipótesis nula? ¿Qué significa?

Se rechaza la hipótesis nula

Eso significa que la beta que calculamos es muy acercada a la real

Realiza otras dos regresiones. Ya tienes hecha la regresión de ventas dado el gasto en publicidad de TV. Realiza la regresión para gastos en radio y gastos en periódico. Organiza las respuestas para que debajo de esta celda se tenga:
- Título de regresión
- Coeficientes estimados
- Errores estándar de los coeficientes
- Intervalos de confianza
- Estadísticos t
- p-values
- Observaciones

### Regresión para gastos en radio

#### Coeficientes estimados

In [31]:
x_radio = datos.radio.values.reshape(-1, 1)
regresion_radio = LinearRegression().fit(x_radio, y)
beta_0_radio = regresion_radio.intercept_
beta_1_radio = regresion_radio.coef_[0]
beta_0_radio, beta_1_radio

(9.311638095158283, 0.20249578339243965)

#### Errores estándar de los coeficientes

In [32]:
RSS_radio = np.sum((y - regresion_radio.predict(x_radio))**2)
var_radio = RSS_radio/(n-2)
denom_radio = np.sum((datos.radio - datos.radio.mean())**2)
SE_beta0_radio = np.sqrt(var_radio*(1/n + (datos.radio.mean()**2)/denom_radio))
SE_beta1_radio = np.sqrt(var_radio/denom_radio)
SE_beta0_radio, SE_beta1_radio

(0.5629004962718711, 0.020411306360090635)

#### Intervalos de confianza

In [33]:
lower_beta0_radio = beta_0_radio - 2*SE_beta0_radio
upper_beta0_radio = beta_0_radio + 2*SE_beta0_radio
lower_beta1_radio = beta_1_radio - 2*SE_beta1_radio
upper_beta1_radio = beta_1_radio + 2*SE_beta1_radio
lower_beta0_radio, upper_beta0_radio, lower_beta1_radio, upper_beta1_radio

(8.185837102614542, 10.437439087702025, 0.1616731706722584, 0.2433183961126209)

#### Estadísticos t

In [34]:
t_0_radio = beta_0_radio / SE_beta0_radio
t_1_radio = beta_1_radio / SE_beta1_radio
t_0_radio, t_1_radio

(16.542245311258217, 9.920765472824959)

#### p-values

In [35]:
P_value_b0_radio = 2*(1 - stats.t.cdf(np.abs(t_0_radio), df=n-2))
P_value_b1_radio = 2*(1 - stats.t.cdf(np.abs(t_1_radio), df=n-2))
P_value_b0_radio, P_value_b1_radio

(0.0, 0.0)

#### Observaciones

Se nota claramente que la hipótesis "No hay efecto" es falsa, por lo que, primeramente, podemos decir que sí hay efecto. 
Segundo, podemos decir que las betas que calculamos son MUY cercanas a las reales

### Regresión para gastos en periódico

#### Coeficientes estimados

In [36]:
x_newspaper = datos.newspaper.values.reshape(-1, 1)
regresion_newspaper = LinearRegression().fit(x_newspaper, y)
beta_0_newspaper = regresion_newspaper.intercept_
beta_1_newspaper = regresion_newspaper.coef_[0]
beta_0_newspaper, beta_1_newspaper

(12.351407069278162, 0.05469309847227335)

#### Errores estándar de los coeficientes

In [37]:
RSS_newspaper = np.sum((y - regresion_newspaper.predict(x_newspaper))**2)
var_newspaper = RSS_newspaper/(n-2)
denom_newspaper = np.sum((datos.newspaper - datos.newspaper.mean())**2)
SE_beta0_newspaper = np.sqrt(var_newspaper*(1/n + (datos.newspaper.mean()**2)/denom_newspaper))
SE_beta1_newspaper = np.sqrt(var_newspaper/denom_newspaper)
SE_beta0_newspaper, SE_beta1_newspaper

(0.6214201876094345, 0.016575721876358165)

#### Intervalos de confianza

In [38]:
lower_beta0_newspaper = beta_0_newspaper - 2*SE_beta0_newspaper
upper_beta0_newspaper = beta_0_newspaper + 2*SE_beta0_newspaper
lower_beta1_newspaper = beta_1_newspaper - 2*SE_beta1_newspaper
upper_beta1_newspaper = beta_1_newspaper + 2*SE_beta1_newspaper
lower_beta0_newspaper, upper_beta0_newspaper, lower_beta1_newspaper, upper_beta1_newspaper

(11.108566694059293,
 13.59424744449703,
 0.02154165471955702,
 0.08784454222498968)

#### Estadísticos t

In [39]:
t_0_newspaper = beta_0_newspaper / SE_beta0_newspaper
t_1_newspaper = beta_1_newspaper / SE_beta1_newspaper
t_0_newspaper, t_1_newspaper

(19.87609562024895, 3.2995907436334178)

#### p-values

In [40]:
P_value_b0_newspaper = 2*(1 - stats.t.cdf(np.abs(t_0_newspaper), df=n-2))
P_value_b1_newspaper = 2*(1 - stats.t.cdf(np.abs(t_1_newspaper), df=n-2))
P_value_b0_newspaper, P_value_b1_newspaper

(0.0, 0.0011481958688881466)

#### Observaciones

En este caso podemos también notar que con seguridad concluimos lo mismo que para radio. Con la única observación de que el beta 1 no es tan segurísimo como el pasado (Pero aún muy seguro). El beta1 nos dice que hace super poco efecto en las ventas el presupuesto de periódico.

## Regresión lineal múltiple

En lugar de hacer una regresión para cada factor independiente, quizás se puede extender el modelo para que tenga varios factores dentro:

$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p + \epsilon $$

Para nuestro ejemplo de publicidad:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

Utiliza la librería `statsmodels` para realizar la regresión. Por defecto la librería statsmodels no toma en cuenta el intercepto ($\beta_0$), por lo que se tendrá que agregar una columna de unos de tamaño *n* a la matriz X.

`import statsmodels.api as sm`

`ols = sm.OLS(Y, X)`

`results = ols.fit()`

`results.summary()`

In [49]:
import statsmodels.api as sm
ones = np.ones((n, 1))
Xbig = np.hstack((ones, X, x_radio, x_newspaper, X*x_radio, X*x_newspaper, x_radio*x_newspaper))

In [50]:
ols = sm.OLS(y, Xbig)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     993.3
Date:                Thu, 28 Aug 2025   Prob (F-statistic):          3.68e-142
Time:                        17:42:50   Log-Likelihood:                -267.49
No. Observations:                 200   AIC:                             549.0
Df Residuals:                     193   BIC:                             572.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4602      0.318     20.342      0.000       5.834       7.087
x1             0.0203      0.002     12.633      0.000       0.017       0.024
x2             0.0229      0.011      2.009      0.046       0.000       0.045
x3             0.0170      0.010      1.691      0.092      -0.003       0.037
x4             0.0011   5.72e-05     19.930      0.000       0.001       0.001
x5         -7.971e-05   3.58e-05     -2.227      0.027      -0.000   -9.12e-06
x6            -0.0001      0.000     -0.464      0.643      -0.001       0.000
==============================================================================
Omnibus:                      111.385   Durbin-Watson:                   2.222
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              767.483
Skew:                          -2.046   Prob(JB):                    2.20e-167
Kurtosis:                      11.681   Cond. No.                     3.80e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.8e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

¿Qué diferencias puedes observar entre los *p-values* de una regresión múltiple y los encontrados en las regresiones simples? ¿Por qué crees que existen estas diferencias?

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3

## Addendum

Para calcular los *p-values* de los parámetros sin `statsmodels`:
1. Calcular RSS
2. Calcular RSE
3. `var_beta = np.linalg.inv(X.T @ X) * rse**2` (X con columna de intercepto)
4. `std_beta = np.sqrt(var_beta.diagonal())` El orden de los valores corresponde al orden de los factores en las columnas de la matriz $X$.
5. Calcular *estadístico t*
6. Calcular *p-value*